In [1]:
import re

In [4]:
content = "Hello World"

In [6]:
print(re.search("W",content)) #search : 문자열 전체 검색

<_sre.SRE_Match object; span=(6, 7), match='W'>


In [32]:
print(re.match("W", content)) #문자열 처음부터 검색

None


In [33]:
data = """
park 800905-1049118
kim 700905-1059119
"""

In [35]:
result=[]
for line in data.split("\n"):
    word_result=[]
    for word in line.split(" "):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + "-" + "*******"
        word_result.append(word)
    result.append(" ".join(word_result))
print("\n".join(result))


park 800905-*******
kim 700905-*******



In [44]:
pat = re.compile("(\d{6}_[-]\d{7})")
print(pat.sub("\g<1>-*******",data))


park 800905-1049118
kim 700905-1059119



In [50]:
p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<_sre.SRE_Match object; span=(0, 4), match='Crow'>


In [53]:
print(re.search('^Life','Life is too short'))
print(re.search('^Life','My Life'))

<_sre.SRE_Match object; span=(0, 4), match='Life'>
None


In [55]:
print(re.search('short$', 'Life is too short'))
print(re.search('short$', 'Life is too short, you need python'))

<_sre.SRE_Match object; span=(12, 17), match='short'>
None


In [56]:
p = re.compile('(ABC)+')

In [57]:
m = p.search('ABCABCABC OK?')

In [58]:
print(m.group())

ABCABCABC


In [59]:
p = re.compile(r'\bclass\b')

In [63]:
print(p.search('no class at all'))
print(p.search('one subclass is'))
print(p.search('the declassified algorithm'))

<_sre.SRE_Match object; span=(3, 8), match='class'>
None
None


In [87]:
p = re.compile(r"\w+\w+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")
print(m.group())

010-1234-1234


In [91]:
p = re.compile("r(\w+)\s+\d+[-]\d+[-]\d+") # r : raw string.. 백슬래시 살려둠 파이썬은 원래 생략시킨단다

In [92]:
m = p.search("park 010-1234-1234")

In [93]:
print(m.group(1))

k


In [94]:
data = "정규식 너무 싫다"

In [100]:
p = re.compile(r"\W")

In [101]:
re.split(p, data)

['정규식', '너무', '싫다']

-------------------------------

# Scraping

In [102]:
import requests

In [107]:
resp = requests.get("http://example.webscraping.com/places/default/search")

In [108]:
from bs4 import BeautifulSoup

In [110]:
dom = BeautifulSoup(resp.content, "lxml")

In [111]:
result = dom.select("#results")

In [112]:
len(result)

1

In [113]:
result

[<div id="results">
 </div>]

In [114]:
#왜 비었냐..?? 비동기식(Ajax 같은거인듯..)으로 가져오는 거는 html파서로 못가져온다

# 그래서 Selenium 이란걸 쓴다

In [117]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.


In [124]:
from selenium import webdriver

In [157]:
driver = webdriver.Chrome("C:/Users/hangs/AppData/Local/Programs/Python/chromedriver.exe")

In [126]:
driver.get("http://example.webscraping.com/places/default/search")

In [128]:
driver.find_element_by_id("search_term").send_keys("korea")

In [129]:
driver.find_element_by_id("search").click()

In [130]:
driver.implicitly_wait(1) # 이건 뭐야 왜해??

In [131]:
links = driver.find_elements_by_css_selector("#results a")

In [132]:
[link.text for link in links]

['North Korea', 'South Korea']

> ### 심심한데 list 내포 공부..
> [출력표현식 for 요소 in iterable [if 조건식]]
> 뭐 많은데.. 대충 알겠음  
> 외워야할거 [[0]*5 for _ in range(5)] : 0인 5*5 배열 짜기  
> [0 for _ in range(5) for _ in range(5)] 이거 아님! 이거는 25개 1차원 리스트 생성

In [201]:
[[i]*5 for i in range(5) if i == 0]

[[0, 0, 0, 0, 0]]

In [141]:
len([0 for _ in range(5) for _ in range(5)])

25

----------------------------

## 네이버 내 메일 목록 등등 긁어오기

In [165]:
%%writefile account.json
    {"id" : "dimes12", "pw": "1111"}

Overwriting account.json


In [166]:
import json

In [167]:
with open("account.json", "r") as f:
    a = json.load(f)

In [168]:
driver.get("http://www.naver.com")

In [183]:
driver.find_element_by_id("id").send_keys(a["id"])

In [184]:
driver.find_element_by_id("pw").send_keys(a["pw"])

In [171]:
driver.find_element_by_css_selector(".btn_login > input").click()

In [174]:
# 등록하라는 창이 뜨면 등록시키는데 난 안뜨는데.. 걍 로그인 됨
#driver.find_element_by_css_selector(".btn_upload > a").click()
#driver.find_element_by_css_selector(".btn_maintain > a").click()

In [175]:
from selenium.common.exceptions import NoSuchElementException

In [185]:
# 그래서 try except 처리 함.. 메일까지 ㄱㄱ
driver.find_element_by_css_selector(".btn_login > input").click()
try : 
    driver.find_element_by_css_selector(".btn_upload > a").click()
    driver.find_element_by_css_selector(".btn_maintain > a").click()
except NoSuchElementException as e:
    print(e)
finally:
    driver.get("https://mail.naver.com/?n=1531981403191&v=f")

Message: no such element: Unable to locate element: {"method":"css selector","selector":".btn_upload > a"}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)



In [186]:
html = BeautifulSoup(driver.page_source, "lxml")

In [187]:
tempList = html.select(".subject strong")

In [194]:
for temp in tempList:
    print(temp.contents[1])

인천국제공항공사 / 인텔리안테크놀로지스 / LG이노텍 / CJ헬스케어 등 시선집중 신입채용 좋은 일 모음!!
#크라운제과 #우리은행 #귀뚜라미 #가장빠른 #채용속보
#LG #SK #삼성 #현대 #대기업 #채용 #취업 #화이팅 
[DBGuide.net] 김기창의 데이터 모델링 강의(17회) : 엔터티 유형에 관한 사색
(광고) 오늘의 급등주는? 동아일보 주식정보업 최초 브랜드대상 수상 기념!
The Ultimate Back-To-School Supply
한화L&C / 태광실업 / 아이센스/ 고려해운항공 등 시선집중 신입채용 좋은 일 모음!!
(광고) 김시순 님 서민을 위한 맞춤대출 그리고 맞춤지원 받으세요.~
인턴으로 업계에 첫 발 내딛자! 한국도로공사 / 한국마사회 / KT 등 채용정보 확인하세요!!
(광고) 오창석 님 1년안에 1억만들기 이것만 알고있으면 가능하다?~(무료정보제공)
(광고)[톡톡편지] 나는 나를 사랑할 수 있을까?
(광고) 연봉 4,000만원 이상! 전세계 1위 위성통신 기업, 인텔리안테크놀로지스 하반기 대규모 채용 시작!
(광고) 2018년 자동차 리서치 참여 부탁드립니다.~ (총5200만원상당 현금, 항공권 추첨)
#속보 #인천국제공항  #CJ #채용 #취업 
2018 BIGDATA X CAMPUS 교육생 의견 청취 및 자격검정 접수 안내
